# Simple cancer binary classification

## Imports

In [1]:
from sklearn.datasets import load_breast_cancer

from neuralnetlib.preprocessing import StandardScaler
from neuralnetlib.activations import Sigmoid, ReLU
from neuralnetlib.layers import Input, Activation, Dense
from neuralnetlib.losses import BinaryCrossentropy
from neuralnetlib.model import Model
from neuralnetlib.optimizers import Adam
from neuralnetlib.metrics import accuracy_score, f1_score, recall_score, precision_score
from neuralnetlib.utils import train_test_split

## 1. Loading a dataset (in this case, Breast Cancer dataset)

In [2]:
data = load_breast_cancer()
X, y = data.data, data.target

## 2. Preprocessing

In [3]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

## 3. Model definition

In [4]:
input_neurons = x_train.shape[1:][0]  # Cancer dataset has 30 features
num_hidden_layers = 5  # Number of hidden layers
hidden_neurons = 100  # Number of neurons in each hidden layer
output_neurons = 1  # Binary classification-regression

model = Model()
model.add(Input(input_neurons))
model.add(Dense(hidden_neurons, weights_init='he', random_state=42))
model.add(Activation(ReLU()))

for _ in range(num_hidden_layers - 1):
    model.add(Dense(hidden_neurons, weights_init='he', random_state=42))
    model.add(Activation(ReLU()))

model.add(Dense(output_neurons, random_state=42))
model.add(Activation(Sigmoid()))

ValueError: The first layer must be an Input layer.

## 4. Model compilation

In [ ]:
model.compile(loss_function=BinaryCrossentropy(), optimizer=Adam(learning_rate=0.0001))

model.summary()

## 5. Model training

In [ ]:
model.fit(x_train, y_train, epochs=20, batch_size=48, metrics=[accuracy_score], random_state=42)

## 6. Model evaluation

In [ ]:
loss = model.evaluate(x_test, y_test)
print(f'Test loss: {loss}')

## 7. Model prediction

In [ ]:
y_pred = model.predict(x_test)

## 8. Printing some metrics

In [ ]:
accuracy = accuracy_score(y_pred, y_test)
precision = precision_score(y_pred, y_test)
recall = recall_score(y_pred, y_test)
f1 = f1_score(y_pred, y_test)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")